In [1]:
!pip install nibabel

In [2]:
import pandas as pd

metadata = '/kaggle/input/mri-images/ADNI1_Complete_1Yr_1.5T_6_20_2025.csv'
meta = pd.read_csv(metadata)
meta.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I112538,941_S_1311,MCI,M,70,m12,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,6/01/2008,NiFTI,NaN
1,I97341,941_S_1311,MCI,M,70,m06,MRI,MPR-R; GradWarp; B1 Correction; N3; Scaled,Processed,9/27/2007,NiFTI,NaN
2,I97327,941_S_1311,MCI,M,69,sc,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,3/02/2007,NiFTI,NaN
3,I75150,941_S_1202,CN,M,78,m06,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,8/24/2007,NiFTI,NaN
4,I105437,941_S_1202,CN,M,79,m12,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,2/28/2008,NiFTI,NaN


In [3]:
# meta.iloc[:, 1].shape

for name in range(len(meta.iloc[:, 1])):
    print(meta.iloc[name, 1])
    break

941_S_1311


In [4]:
import os

data_dir = '/kaggle/input/mri-preprocessed/pre_processed_files1'

files = [f for f in os.listdir(data_dir) if f.endswith('.nii')]

print(len(files))

445


In [5]:
file_ids = []
for file in files:
    parts = file.split('_')
    file_str = parts[2] + '_' + parts[3] + '_' + parts[4]
    file_ids.append(file_str)

# len(file_ids)

y = []
# for subject in range(len(meta.iloc[:, 1])):
#     for file_id in file_ids:
#         if (file_id == meta.iloc[subject, 1]):
#             y.append([meta.iloc[subject, 1], meta.iloc[subject, 2]])

for file_id in file_ids:
    for subject in range(len(meta.iloc[:, 1])):
        if (file_id == meta.iloc[subject, 1]):
            y.append([meta.iloc[subject, 1], meta.iloc[subject, 2]])
            break
        continue


In [6]:
file_ids[:5], y[:5]
# file_ids contain the id of the preprocessed files
# y contains the [id, target] for them

(['023_S_0887', '136_S_0300', '022_S_1394', '005_S_0610', '036_S_0813'],
 [['023_S_0887', 'MCI'],
  ['136_S_0300', 'AD'],
  ['022_S_1394', 'MCI'],
  ['005_S_0610', 'CN'],
  ['036_S_0813', 'CN']])

In [7]:
y[10]

['029_S_1215', 'MCI']

In [ ]:
# visualising data
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt


nii_file = os.path.join(data_dir, files[10])
img = nib.load(nii_file)
data = img.get_fdata()

print("Shape:", data.shape)

def show_slices(slices):
    fig, axes = plt.subplots(1, len(slices), figsize=(15, 5))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")
    plt.tight_layout()
    plt.show()

# Select central slices from axial, sagittal, coronal planes
sagittal = data[data.shape[0] // 2, :, :]
coronal  = data[:, data.shape[1] // 2, :]
axial    = data[:, :, data.shape[2] // 2]

show_slices([sagittal, coronal, axial])

## Data

In [8]:
import nibabel as nib
import os
import numpy as np
import matplotlib.pyplot as plt

# Directory path
data_dir = "/kaggle/input/mri-preprocessed/pre_processed_files1"
nii_list = os.listdir(data_dir)

max_axial_shape = (0, 0)

for filename in nii_list:
    path = os.path.join(data_dir, filename)
    nii_img = nib.load(path)
    data = nii_img.get_fdata()
    
    axial_shape = data.shape[:2]

    if np.prod(axial_shape) > np.prod(max_axial_shape):
        max_axial_shape = axial_shape

print(f"Axial plane shape: {max_axial_shape}")

Axial plane shape: (184, 256)


In [9]:
def pad_to_shape(img, target_shape):
    h, w = img.shape
    H, W = target_shape
    pad_top = (H - h) // 2
    pad_bottom = H - h - pad_top
    pad_left = (W - w) // 2
    pad_right = W - w - pad_left
    return np.pad(img, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant')


padded_images = []
y_ordered = []


for file in os.listdir(data_dir):
    if file.endswith('.nii') or file.endswith('.nii.gz'):
        parts = file.split('_')
        file_str = parts[2] + '_' + parts[3] + '_' + parts[4]

        for i in range(len(y)):
            if file_str == y[i][0]:
                y_ordered.append([file_str, y[i][1]])
                break
    
        nii_path = os.path.join(data_dir, file)
        data = nib.load(nii_path).get_fdata()
        
        axial = data[:, :, data.shape[2] // 2]    # axial central slice
        padded = pad_to_shape(axial, max_axial_shape)
        
        padded_images.append(padded)
        # labels.append(label_map[label])

X = np.array(padded_images).reshape(-1, max_axial_shape[0], max_axial_shape[1], 1)

print(f"X shape: {X.shape}")

X shape: (445, 184, 256, 1)


In [10]:
len(y_ordered)

445

In [11]:
# label encodings

y_data = []
for id_, label in y_ordered:
    if label == 'MCI':
        y_data.append(1)
    elif label == 'AD':
        y_data.append(2)
    elif label == 'CN':
        y_data.append(0)
    else:
        print(f"Label: {label}")
        
y_data = np.array(y_data)
y_data.shape

(445,)

In [ ]:
# y_ordered -> contains [image_id, label]
# X -> (445, 184, 256, 1) axial slices
# y_data -> contains labels in the same order as the X values {0, 1, 2}

## Model

In [12]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [13]:
class cnn1(nn.Module):
    def __init__(self, kernel_size=3, dropout_rate=0.2, in_channels=1):
        super(cnn1, self).__init__()

        self.K = kernel_size
        self.in_channels = in_channels
        self.dropout_rate = dropout_rate

        self.net = nn.Sequential(
            nn.Conv2d(self.in_channels, 16, kernel_size=self.K, padding=1),
            nn.ReLU(),

            nn.Conv2d(16, 16, kernel_size=self.K, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(16, 64, kernel_size=self.K, padding=1),
            nn.ReLU(),

            nn.Conv2d(64, 64, kernel_size=self.K, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 256, kernel_size=self.K, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout2d(self.dropout_rate),
            nn.Flatten(),
            nn.Linear(188416, 128) # TODO: Check this line
        )

    def forward(self, x):
        x = self.net(x)
        return x


In [14]:
class cnn2(nn.Module):
    def __init__(self, kernel_size=5, dropout_rate=0.2, in_channels=1):
        super(cnn2, self).__init__()

        self.K = kernel_size
        self.in_channels = in_channels
        self.dropout_rate = dropout_rate

        self.net = nn.Sequential(
            nn.Conv2d(self.in_channels, 32, kernel_size=self.K, padding=1),
            nn.ReLU(),

            nn.Conv2d(32, 32, kernel_size=self.K, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 128, kernel_size=self.K, padding=1),
            nn.ReLU(),

            nn.Conv2d(128, 128, kernel_size=self.K, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 512, kernel_size=self.K, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout2d(self.dropout_rate),
            nn.Flatten(),
            nn.Linear(296960, 128)
        )

    def forward(self, x):
        x = self.net(x)
        return x


In [15]:
class ADNet(nn.Module):
    def __init__(self, input_dim=224, dropout_rate=0.2, in_channels=1, n_labels=3):
        super(ADNet, self).__init__()

        self.input_dim = input_dim
        self.dropout_rate = dropout_rate
        self.in_channels = in_channels
        self.n_labels = n_labels

        self.cnn1 = cnn1(dropout_rate=self.dropout_rate, in_channels=self.in_channels)
        self.cnn2 = cnn2(dropout_rate=self.dropout_rate, in_channels=self.in_channels)

        self.classifier = nn.Sequential(
            nn.Linear(128 + 128, 128),
            nn.ReLU(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(128, self.n_labels)   # final layer for binary classification
        )

    def forward(self, x):
        x1 = self.cnn1(x)   # shape: (B, 128)
        x2 = self.cnn2(x)   # shape: (B, 128)

        x = torch.cat((x1, x2), dim=1)  # concatenate features from both CNNs, shape: (B, 256)
        x = self.classifier(x)          # shape: (B, 1)
        return x


In [16]:
# testing dimensions
model = ADNet()
input_tensor = torch.randn(8, 1, 184, 256)  # batch of 8 single-channel images
output = model(input_tensor)               # output shape: (8, n_labels)
print(output.shape)  # should print torch.Size([8, 1]) for binary classification
print(model)  # print the model architecture

torch.Size([8, 3])
ADNet(
  (cnn1): cnn1(
    (net): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU()
      (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU()
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU()
      (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (14): ReLU()
      (15): Dropout2d(p=0.2, inplace=False)
      (16): Flatten(start_dim=1, end_dim=-1

In [19]:
# dataset class
from torch.utils.data import Dataset

class mri_data(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int64)

        # Reshape: (N, H, W, 1) → (N, 1, H, W) for PyTorch conv layers
        if self.X.ndim == 4 and self.X.shape[-1] == 1:
            self.X = np.transpose(self.X, (0, 3, 1, 2))  # (N, 1, H, W)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img = torch.from_numpy(self.X[idx])
        label = torch.tensor(self.y[idx], dtype=torch.long)
        return img, label


In [20]:
from torch.utils.data import DataLoader, random_split

# Wrap into Dataset
dataset = mri_data(X, y_data)

# Split into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

# DataLoaders
train_loader = DataLoader(train_ds, batch_size=3, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=3, shuffle=False)

In [21]:
# training
model = ADNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [30]:
# train loop
def train_epoch(model, dataloader, optimizer, criterion):
  model.train()
  current_loss = 0.0
  correct, total = 0, 0

  for inputs, targets in dataloader:
    inputs, targets = inputs.to(device), targets.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    loss.backward()
    optimizer.step()

    current_loss += loss.item() * inputs.size(0)
    _, predicted = outputs.max(1)
    correct += predicted.eq(targets).sum().item()
    total += targets.size(0)

  avg_loss = current_loss / total
  accuracy = correct / total

  return avg_loss, accuracy

In [32]:
# evaluation loop
def evaluate(model, dataloader, criterion):
  model.eval()
  current_loss = 0.0
  correct, total = 0, 0

  with torch.no_grad():
    for inputs, targets in dataloader:
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, targets)

      current_loss += loss.item() * inputs.size(0)
      _, predicted = outputs.max(1)
      correct += predicted.eq(targets).sum().item()
      total += targets.size(0)

  avg_loss = current_loss / total
  accuracy = correct / total

  return avg_loss, accuracy

In [24]:
# training loop
def train_model(model, train_loader, val_loader, epochs=10):
  for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc*100:.2f}%")
    print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc*100:.2f}%\n")

In [33]:
train_model(model, train_loader, val_loader, epochs=10)

Epoch 1/10
  Train Loss: 1.0350, Accuracy: 53.09%
  Val   Loss: 1.3111, Accuracy: 46.07%

Epoch 2/10
  Train Loss: 1.0040, Accuracy: 57.58%
  Val   Loss: 1.5778, Accuracy: 48.31%

Epoch 3/10
  Train Loss: 0.8402, Accuracy: 63.76%
  Val   Loss: 1.4986, Accuracy: 48.31%

Epoch 4/10
  Train Loss: 0.6551, Accuracy: 75.00%
  Val   Loss: 1.3711, Accuracy: 48.31%

Epoch 5/10
  Train Loss: 0.4651, Accuracy: 82.58%
  Val   Loss: 1.8211, Accuracy: 40.45%

Epoch 6/10
  Train Loss: 0.3552, Accuracy: 87.92%
  Val   Loss: 1.6741, Accuracy: 47.19%

Epoch 7/10
  Train Loss: 0.1522, Accuracy: 95.51%
  Val   Loss: 2.3766, Accuracy: 43.82%

Epoch 8/10
  Train Loss: 0.1334, Accuracy: 94.38%
  Val   Loss: 2.1093, Accuracy: 42.70%

Epoch 9/10
  Train Loss: 0.0600, Accuracy: 98.88%
  Val   Loss: 2.4408, Accuracy: 46.07%

Epoch 10/10
  Train Loss: 0.0390, Accuracy: 98.88%
  Val   Loss: 2.7412, Accuracy: 47.19%

